<a href="https://colab.research.google.com/github/kyoungkeun-lee/WHS2/blob/main/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import pandas as pd
import json
from datetime import datetime

# 파일 경로 설정
file_path = '/content/drive/MyDrive/Colab Notebooks/dataset/nsl-kdd/data.txt'

# 파일 읽기
with open(file_path, 'r') as file:
    lines = file.readlines()

# 각 행을 개별적으로 파싱
parsed_data = []
for line in lines:
    try:
        parsed_data.append(json.loads(line))
    except json.JSONDecodeError as e:
        print(f"Error parsing line: {line}")
        print(f"Error: {e}")

# JSON 데이터를 데이터프레임으로 변환
df = pd.json_normalize(parsed_data)

# timestamp를 변환하는 함수 정의
def convert_epoch_to_time(epoch):
    # Epoch 시간을 datetime 객체로 변환
    dt = datetime.fromtimestamp(epoch / 1000.0)
    # 시간을 '시:분' 형식으로 변환하여 반환
    return dt.strftime('%H:%M')

# timestamp 칼럼 변환
df['timestamp'] = df['timestamp'].apply(convert_epoch_to_time)

# 정리된 데이터프레임 확인
print(df.head())

# 정리된 데이터프레임을 CSV 파일로 저장
df.to_csv('/content/drive/MyDrive/Colab Notebooks/dataset/nsl-kdd/TestFinal.csv', index=False)


Error parsing line: 

Error: Expecting value: line 2 column 1 (char 1)
Error parsing line: 

Error: Expecting value: line 2 column 1 (char 1)
Error parsing line: 

Error: Expecting value: line 2 column 1 (char 1)
Error parsing line: 

Error: Expecting value: line 2 column 1 (char 1)
Error parsing line: 

Error: Expecting value: line 2 column 1 (char 1)
Error parsing line: 

Error: Expecting value: line 2 column 1 (char 1)
Error parsing line: 

Error: Expecting value: line 2 column 1 (char 1)
Error parsing line: 

Error: Expecting value: line 2 column 1 (char 1)
Error parsing line: 

Error: Expecting value: line 2 column 1 (char 1)
Error parsing line: 

Error: Expecting value: line 2 column 1 (char 1)
Error parsing line: 

Error: Expecting value: line 2 column 1 (char 1)
Error parsing line: 

Error: Expecting value: line 2 column 1 (char 1)
Error parsing line: 

Error: Expecting value: line 2 column 1 (char 1)
Error parsing line: 

Error: Expecting value: line 2 column 1 (char 1)
Error 

In [83]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib

# Load the dataset
file_path = '/content/drive/MyDrive/Colab Notebooks/dataset/nsl-kdd/TestFinal.csv'
data = pd.read_csv(file_path)

# Drop columns that are not useful for modeling
columns_to_drop = ['webaclId', 'terminatingRuleId', 'terminatingRuleType',
                   'terminatingRuleMatchDetails', 'httpSourceName', 'httpSourceId',
                   'ruleGroupList', 'rateBasedRuleList', 'nonTerminatingMatchingRules',
                   'excludedRules', 'httpRequest.headers', 'httpRequest.uri',
                   'httpRequest.args', 'httpRequest.httpVersion', 'httpRequest.requestId']

actual_columns_to_drop = [col for col in columns_to_drop if col in data.columns]
data = data.drop(columns=actual_columns_to_drop)

# Drop columns with persistent missing values
columns_with_missing_values = ['requestHeadersInserted', 'responseCodeSent']
data = data.drop(columns=columns_with_missing_values)

# Encode categorical variables before handling missing values
label_encoders = {}
categorical_columns = ['action', 'httpRequest.clientIp', 'httpRequest.country', 'httpRequest.httpMethod','timestamp']

for column in categorical_columns:
    if column in data.columns:
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column])
        label_encoders[column] = le

# Handle missing values by filling with the median for numeric columns and the most frequent value for categorical columns
for column in data.columns:
    if data[column].dtype in [int, float]:
        data[column].fillna(data[column].median(), inplace=True)
    else:
        data[column].fillna(data[column].mode()[0], inplace=True)

# Check for any remaining missing values
missing_values = data.isnull().sum()
print("Missing values per column after imputation:")
print(missing_values[missing_values > 0])

# Verify there are no more NaN values
assert data.isnull().sum().sum() == 0, "There are still missing values in the dataset"

# Separate features and target
X = data.drop(columns=['action'])
y = data['action']

# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Save the preprocessed data for further steps
X_scaled_filename = 'X_scaled.joblib'
y_filename = 'y.joblib'
joblib.dump(X_scaled, X_scaled_filename)
joblib.dump(y, y_filename)

# Confirm the files have been saved
X_scaled_filename, y_filename
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import xgboost as xgb

# Load the preprocessed data
X_scaled_filename = 'X_scaled.joblib'
y_filename = 'y.joblib'
X_scaled = joblib.load(X_scaled_filename)
y = joblib.load(y_filename)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Set the parameters for the XGBoost model to speed up training
params = {
    'n_estimators': 50,            # Further reduce the number of boosting rounds
    'max_depth': 3,                # Reduce the maximum depth of trees
    'learning_rate': 0.1,          # Set the learning rate
    'eval_metric': 'logloss',      # Evaluation metric
    'use_label_encoder': False,    # Disable label encoder usage
    'early_stopping_rounds': 10,   # Early stopping
}

# Use a smaller subset of the data for quick evaluation
X_train_subset, _, y_train_subset, _ = train_test_split(X_train, y_train, train_size=0.1, random_state=42)

# Train the XGBoost model with the specified parameters on the subset
model = xgb.XGBClassifier(**params)
model.fit(X_train_subset, y_train_subset, eval_set=[(X_test, y_test)], verbose=False)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred, average='binary')

# Save the trained model
model_filename = '/content/drive/MyDrive/Colab Notebooks/dataset/nsl-kdd/xgboost_model.joblib'
joblib.dump(model, model_filename)

f1, model_filename

Missing values per column after imputation:
Series([], dtype: int64)


(0.9262394195888755,
 '/content/drive/MyDrive/Colab Notebooks/dataset/nsl-kdd/xgboost_model.joblib')